In [35]:
import functools
import pickle
import pandas as pd

def partialclass(cls, *args, **kwds):

    NewCls = type('_' + cls.__name__, (cls,),
                  {'__init__': functools.partialmethod(cls.__init__, *args, **kwds),
                   '__module__': __name__,
                   'parent_class': cls})

    return NewCls

In [27]:
class TestClass:
    def __init__(self, print_str, parent_class=None):
        self.print_str = print_str
        try:
            getattr(self, '_parent_class')
        except AttributeError:
            self._parent_class = self.__class__
    
    def print_func(self):
        print(self.print_str)
        
    
    def __reduce__(self):
        return (self._parent_class, (self.print_str ),) )
'''    
    def __call__(self, print_str):
        if self.__class__ is not self._parent_class:
            return self.__class__(print_str)
'''

'    \n    def __call__(self, print_str):\n        if self.__class__ is not self._parent_class:\n            return self.__class__(print_str)\n'

In [28]:
test = TestClass('string')
new_test = partialclass(type(test), print_str=test.print_str)()

In [29]:
type(new_test)

__main__._TestClass

In [30]:
new_test._parent_class

__main__.TestClass

In [31]:
data = pickle.dumps(new_test)

In [32]:
load_test = pickle.loads(data)
load_test.print_func()

('print_str', 'string')


In [33]:
type(load_test)

__main__.TestClass

In [61]:
class DataFrameClass(pd.DataFrame):
    _metadata = pd.DataFrame._metadata + ['kwds']
    
    def __init__(self, data=None, index=None, columns=None, dtype=None, copy=False, **kwds):
        self.kwds = kwds
        super().__init__(data, index, columns, dtype, copy)
        print(self.__dict__)
        if 'parent_class' not in type(self).__dict__:
            print('new _parent_class')
            setattr(self.__class__, 'parent_class', type(self))
            
    @property
    def _constructor(self):
        print('_constructor')
        print('0: ', type(self).parent_class)
        return partialclass(type(self), parent_class=type(self).parent_class, **self.kwds)


In [62]:
df = DataFrameClass([])
df_copy = df.copy()

{'kwds': {}, '_is_copy': None, '_data': BlockManager
Items: Index([], dtype='object')
Axis 1: Index([], dtype='object'), '_item_cache': {}}
new _parent_class
_constructor
0:  <class '__main__.DataFrameClass'>
{'kwds': {'parent_class': <class '__main__.DataFrameClass'>}, '_is_copy': None, '_data': BlockManager
Items: Index([], dtype='object')
Axis 1: Index([], dtype='object'), '_item_cache': {}}


In [64]:
df_copy.parent_class

__main__.DataFrameClass